Nouveaux concepts : 
-

- Utiliser les mots les tokens en entrée du réseau, en apprenant un embedding, comme ça on rentre tout d'un coup (toutes les infos)

- PPO + LSTM/Transformer comme nouvelle architecture, Q-learning c'est pas ouf car ça nécessite de connaitre tous les states


In [53]:
#1 on se place 
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [54]:
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
import json
from poke_env.environment.pokemon import Pokemon
import poke_env
import pandas as pd
import numpy as np
from poke_env.data import GenData
from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.env_util import make_vec_env
from IPython.display import display
import ipywidgets as widgets
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
import torch
from gymnasium.spaces import Space, Box, Discrete

In [55]:
p = Pokemon(gen=4,species= 'absol')
print(p.base_stats)
print(p.ability)
print(p.item)
print(p.base_stats)

{'atk': 130, 'def': 60, 'hp': 65, 'spa': 75, 'spd': 60, 'spe': 75}
None
unknown_item
{'atk': 130, 'def': 60, 'hp': 65, 'spa': 75, 'spd': 60, 'spe': 75}


Vecteur obs :
-


battle.all_active_pokemons : renvoie les deux pokémons sur le terrain

battle.current_observation : renvoie un sacré truc (détaillé après)

-> Contient les PVs et status de chaque pokémon

Clés de battle.current_observation :
['side_conditions', 'opponent_side_conditions', 'weather', 'fields', 'active_pokemon', 'opponent_active_pokemon', 'team', 'opponent_team', 'events']

events a en majorité rien

In [56]:
from math import floor

In [57]:
#MAJ : tout mettre au format vecteur
#MAJ : ajout du padding 
def ObservedPokemon_to_list(pokemon,is_active_pokemon,is_opponent_team) -> list: 
    # Nom du Pokémon
    species = getattr(pokemon, "species", "UNKNOWN")
    species_id = species_to_id[species]
    item = getattr(pokemon, "item", None)

    item_id = item_to_id[item]
    p = Pokemon(gen=4,species= species)


    # Fraction de vie (float entre 0 et 1)
    hp = getattr(pokemon, "current_hp_fraction", 0.0) or 0.0

    # Types (liste de strings ou [])
    types = [p.type_1.name.lower() if p.type_1 else None, p.type_2.name.lower() if p.type_2 else None]
    if p.type_1 and p.type_2 :
        types = sorted([t for t in types])
    type_str = types[0] if types[1] is None else f"{types[0]}_{types[1]}"
    type_id = types_to_id[type_str]

    # Statut (string ou "NONE")
    status = getattr(pokemon, "status", None)
    status_str = status.name if status else "NONE"
    status_to_id = {
     'NONE': 0,
    'PAR': 1,     # paralysie
    'SLP': 2,     # sommeil
    'BRN': 3,     # brûlure
    'FRZ': 4,     # gel
    'PSN': 5,     # poison
    'TOX': 6,     # poison grave
    'FNT': 7
        }
    status_id = status_to_id[status_str] 

    #Boosts
    if(is_active_pokemon == 1) :

        # Boosts (dict ou [0]*7)
        boosts = getattr(pokemon, "boosts", None)
        boosts_list = list(boosts.values()) if boosts else [0] * 7

    # Stats (dict ou [0]*6)
    if is_opponent_team == 0 : #Si c'est pas l'équipe adverse :

        stats = getattr(pokemon, "stats", None)
        stat_list = list(stats.values()) if stats else [0] * 6
        stats_list = [ np.float32(x / 504) for x in stat_list ]

    else : #si c'est l'équipe adverse, on fait autrement
        level = p.level if hasattr(p, "level") and p.level else 80
        hp_stat = floor(((2 * p.base_stats["hp"] + 31 + 84 // 4) * level) / 100) + level + 10
        if hp == None : 
            hp = 1.0
            #On rajoute un quasi-normalize, pour aller autour de 1 sans que ça soit trop petit, donc on prend pas l'exemple leuphorie 714 stats en PV avec lvl 100, nature, EVs
            
        atk = floor((2 * p.base_stats["atk"] + 31 + 84 // 4) * level / 100 + 5) 
        def_ = floor((2 * p.base_stats["def"] + 31 + 84 // 4) * level / 100 + 5) 
        spa = floor((2 * p.base_stats["spa"] + 31 + 84 // 4) * level / 100 + 5) 
        spd = floor((2 * p.base_stats["spd"] + 31 + 84 // 4) * level / 100 + 5) 
        spe = floor((2 * p.base_stats["spe"] + 31 + 84 // 4) * level / 100 + 5) 
        
        stats_list = [hp_stat,np.float32(atk/504),np.float32(def_/504),np.float32(spa/504),np.float32(spd/504),np.float32(spe/504)]
    #moovepool
    moves_dict = getattr(pokemon, "moves", None)
    move_names = list(moves_dict.keys()) if moves_dict else []

    # Pour avoir toujours 4 valeurs :
    while len(move_names) < 4:
        move_names.append("None")

    move_names_id = [
        moves_to_id[x.lower().replace(" ", "")] if x and x.lower() != "none" else 0 #Padding 0
        for x in move_names
        ]
        
    ability = p.ability
    ability_id = ability_to_id[ability] if ability else 0 # Padding 0 
        
        
    if(is_active_pokemon == 1) :

        return [
            species_id,
            item_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *boosts_list,
            *stats_list,
            *move_names_id[:4]
        ]
    else : 
        return [
            species_id,
            item_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *stats_list,
            *move_names_id[:4]
        ]

In [58]:
#my_team_switches avec les stats, mooves, %hp, status
from itertools import chain

def get_my_switches_with_all_infos(switches,dict) : 

    # Dictionnaire contenant les ObservedPokemon (via battle.current_observation.__dict__["team"])
    team_obs_dict = dict["team"]

    enriched_switches = []

    for poke in switches:
        # Trouver l'ObservedPokemon correspondant en comparant le species
        matched_obs = next(
            (obs for obs in team_obs_dict.values()
             if hasattr(obs, "species") and obs.species == poke.species),
            None
        )

        if matched_obs:
            enriched_switches += ObservedPokemon_to_list(matched_obs,is_active_pokemon=0,is_opponent_team=0)
        else:
            # Fallback si non trouvé
            enriched_switches += ["UNKNOWN", 0.0, "NONE"] + [0]*7 + [0]*6

    return enriched_switches
    
def get_opponent_team_with_all_infos(dict,battle) : 

    # Dictionnaire contenant les ObservedPokemon (via battle.current_observation.__dict__["team"])
    team_obs_dict = dict["opponent_team"]
    #enlever le pokemon actif adverse
    active = battle.opponent_active_pokemon

    opponent_team_list = []
    for poke in team_obs_dict.values() :
        if poke.species != active.species:
            opponent_team_list += ObservedPokemon_to_list(poke,is_active_pokemon=0,is_opponent_team=1)


    return opponent_team_list

In [59]:
####Tests embedding ########

class adapted_obs_debug :
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = list(self.dict_observation["side_conditions"].values())

        self.opponent_side_conditions = list(self.dict_observation["opponent_side_conditions"].values())

        self.weather = list(self.dict_observation["weather"].values())

        self.active_pokemon = ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0)
        self.my_team_switches = get_my_switches_with_all_infos(battle.available_switches,self.dict_observation)
        self.opponent_active_pokemon = ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = get_opponent_team_with_all_infos(self.dict_observation,battle)
        #Fin équipe adverse 
        self.output =   ["side_conditions"] + self.side_conditions + ["opponent_side_conditions"] + self.opponent_side_conditions + ["weather"] + self.weather + ["active_pokemon"] + self.active_pokemon + ["my_team_switches"] + self.my_team_switches + ["opponent_active_pokemon"] + self.opponent_active_pokemon + ["opponent_team"] + self.opponent_team

    #)

Visualizer obs : 
-

**On a tout ce qu'on peut avoir :**

pièges posés (pdr, spikes ..)

climat

notre poké actif avec ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

notre team avec ses pv, talent, ses stats, ses mooves, son nom (species), ses types, ses status

poké adverse avec ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

team adverse avec : ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

--> Il nous manquera juste ce qui a été joué par l'adversaire, pour l'instant je passe là-dessus, amsi ça veut dire que notre IA pourra pas s'adapter en fonction du style du joueur

**Embeddings de :**
-
objets

talents

attaques

noms species

status

types


Aucune ID à 0 pour garder le 0 pour le padding. Tous les dicos commencent à 1 (cf regarder codde en dessous)

1 : Status
- 
juste un idx pour chaque statut

2 : objets
-


In [60]:
item_to_id = {
    "unknown_item": 0,
    None : 0,
    "None": 0,
    '': 0,
    "choiceband": 1,
    "choicescarf": 2,
    "choicespecs": 3,
    "expertbelt": 4,
    "focussash": 5,
    "leftovers": 6,
    "lifeorb": 7,
    "lumberry": 8,
    "powerherb": 9,
    "salacberry": 10,
    "sitrusberry": 11,
    "blacksludge": 12,
    "thickclub": 13,       # os de Ossatueur (Marowak)
    "lightball": 14,       # objet de Pikachu
    "flameplate": 15, 
    "fistplate": 16,
    "earthplate": 17, 
    "stoneplate": 18, 
    "zapplate": 19,
    "meadowplate": 20, 
    "icicleplate": 21, 
    "toxicplate": 22, 
    "mindplate": 23, 
    "insectplate": 24,
    "spookyplate": 25, 
    "dracoplate": 26, 
    "dreadplate": 27, 
    "ironplate": 28, 
    "skyplate": 29,
    "splashplate": 30,
    "damprock": 31,
    "blackglasses": 32,
    "silkscarf": 33,
    "toxicorb": 34,
    "griseousorb" : 35,
    "stick": 36,
    "lustrousorb" : 37,
    "souldew" : 38,
    "chestoberry": 39,
    "custapberry": 40
}

3 : Talents
-

In [61]:
ability_to_id = {
    "adaptability": 1,
    "aftermath": 2,
    "airlock": 3,
    "anticipation": 4,
    "arenatrap": 5,
    "baddreams": 6,
    "battlearmor": 7,
    "blaze": 8,
    "chlorophyll": 9,
    "clearbody": 10,
    "cloudnine": 11,
    "colorchange": 12,
    "compoundeyes": 13,
    "cutecharm": 14,
    "download": 15,
    "drizzle": 16,
    "drought": 17,
    "dryskin": 18,
    "earlybird": 19,
    "filter": 20,
    "flamebody": 21,
    "flashfire": 22,
    "flowergift": 23,
    "forecast": 24,
    "forewarn": 25,
    "frisk": 26,
    "gluttony": 27,
    "guts": 28,
    "hugepower": 29,
    "hydration": 30,
    "hypercutter": 31,
    "immunity": 32,
    "innerfocus": 33,
    "insomnia": 34,
    "intimidate": 35,
    "ironfist": 36,
    "keeneye": 37,
    "leafguard": 38,
    "levitate": 39,
    "limber": 40,
    "liquidooze": 41,
    "magicguard": 42,
    "magnetpull": 43,
    "marvelscale": 44,
    "minus": 45,
    "moldbreaker": 46,
    "motordrive": 47,
    "multitype": 48,
    "naturalcure": 49,
    "noguard": 50,
    "overgrow": 51,
    "owntempo": 52,
    "pickup": 53,
    "plus": 54,
    "poisonheal": 55,
    "poisonpoint": 56,
    "pressure": 57,
    "purepower": 58,
    "quickfeet": 59,
    "rockhead": 60,
    "roughskin": 61,
    "runaway": 62,
    "sandstream": 63,
    "sandveil": 64,
    "scrappy": 65,
    "serenegrace": 66,
    "shadowtag": 67,
    "shedskin": 68,
    "shellarmor": 69,
    "shielddust": 70,
    "simple": 71,
    "skilllink": 72,
    "slowstart": 73,
    "sniper": 74,
    "snowcloak": 75,
    "snowwarning": 76,
    "solidrock": 77,
    "soundproof": 78,
    "speedboost": 79,
    "static": 80,
    "steadfast": 81,
    "stickyhold": 82,
    "sturdy": 83,
    "suctioncups": 84,
    "superluck": 85,
    "swarm": 86,
    "swiftswim": 87,
    "synchronize": 88,
    "tangledfeet": 89,
    "technician": 90,
    "thickfat": 91,
    "tintedlens": 92,
    "torrent": 93,
    "trace": 94,
    "truant": 95,
    "unburden": 96,
    "vitalspirit": 97,
    "voltabsorb": 98,
    "waterabsorb": 99,
    "waterveil": 100,
    "whitesmoke": 101,
    "wonderguard": 102
}

4 : moves 
-

In [62]:
import pandas as pd

df = pd.read_csv("data/csv/move_to_id.csv")  # ou le chemin local
moves_to_id = dict(zip(df["move"], df["id"]))
moves_to_id["hiddenpower"] = 186
for i in moves_to_id.keys() :
    moves_to_id[i] = moves_to_id[i] + 1

In [63]:
print(moves_to_id)

{'aerialace': 1, 'aeroblast': 2, 'agility': 3, 'airslash': 4, 'aquajet': 5, 'aquatail': 6, 'aromatherapy': 7, 'aurasphere': 8, 'batonpass': 9, 'bellydrum': 10, 'bite': 11, 'blizzard': 12, 'bodyslam': 13, 'bravebird': 14, 'brickbreak': 15, 'bugbite': 16, 'bugbuzz': 17, 'bulkup': 18, 'bulletpunch': 19, 'calmmind': 20, 'chatter': 21, 'closecombat': 22, 'counter': 23, 'crabhammer': 24, 'crosschop': 25, 'crunch': 26, 'curse': 27, 'darkpulse': 28, 'darkvoid': 29, 'destinybond': 30, 'discharge': 31, 'doubleedge': 32, 'dracometeor': 33, 'dragonclaw': 34, 'dragondance': 35, 'dragonpulse': 36, 'drainpunch': 37, 'drillpeck': 38, 'dynamicpunch': 39, 'earthpower': 40, 'earthquake': 41, 'encore': 42, 'energyball': 43, 'eruption': 44, 'explosion': 45, 'extremespeed': 46, 'facade': 47, 'fakeout': 48, 'fireblast': 49, 'firefang': 50, 'firepunch': 51, 'flamethrower': 52, 'flareblitz': 53, 'flashcannon': 54, 'focusblast': 55, 'focuspunch': 56, 'gigaimpact': 57, 'glare': 58, 'grassknot': 59, 'gunkshot': 6

5 : Species
-

In [64]:
df = pd.read_csv("data/csv/species_to_id.csv")  # ou le chemin local
species_to_id = dict(zip(df["species"], df["id"]))
for i in species_to_id.keys() :
    species_to_id[i] = species_to_id[i] + 1

In [65]:
species_to_id["gastrodoneast"] = 100
species_to_id["gastrodonwest"] = 100


In [66]:
#Ajout de tous les types de unown
for c in "abcdefghijklmnopqrstuvwxyz":
    species_to_id[f"unown{c}"] = species_to_id["unown"]

In [67]:
species_to_id["unownquestion"] = species_to_id["unown"]
species_to_id["unownexclamation"] = species_to_id["unown"]


In [68]:
print(species_to_id)

{'abomasnow': 1, 'absol': 2, 'aerodactyl': 3, 'aggron': 4, 'alakazam': 5, 'altaria': 6, 'ambipom': 7, 'ampharos': 8, 'arbok': 9, 'arcanine': 10, 'arceus': 11, 'arceusbug': 12, 'arceusdark': 13, 'arceusdragon': 14, 'arceuselectric': 15, 'arceusfighting': 16, 'arceusfire': 17, 'arceusflying': 18, 'arceusghost': 19, 'arceusgrass': 20, 'arceusground': 21, 'arceusice': 22, 'arceuspoison': 23, 'arceuspsychic': 24, 'arceusrock': 25, 'arceussteel': 26, 'arceuswater': 27, 'ariados': 28, 'armaldo': 29, 'articuno': 30, 'azelf': 31, 'azumarill': 32, 'banette': 33, 'bastiodon': 34, 'beautifly': 35, 'beedrill': 36, 'bellossom': 37, 'bibarel': 38, 'blastoise': 39, 'blaziken': 40, 'blissey': 41, 'breloom': 42, 'bronzong': 43, 'butterfree': 44, 'cacturne': 45, 'camerupt': 46, 'carnivine': 47, 'castform': 48, 'celebi': 49, 'charizard': 50, 'chatot': 51, 'cherrim': 52, 'chimecho': 53, 'claydol': 54, 'clefable': 55, 'cloyster': 56, 'corsola': 57, 'cradily': 58, 'crawdaunt': 59, 'cresselia': 60, 'crobat': 

6 : Types
-

In [69]:
# Liste des types Gen 4
types = [
    "bug", "dark", "dragon", "electric", "fighting", "fire", "flying", "ghost",
    "grass", "ground", "ice", "normal", "poison", "psychic", "rock", "steel",
    "water"
]

# Générer les combinaisons mono- et duo-types, sans doublons (ordre alphabétique)
type_combos = set()

for t in types:
    type_combos.add((t,))
    for t2 in types:
        if t != t2:
            type_combos.add(tuple(sorted([t, t2])))

# Convertir en string clés
combo_keys = ["_".join(combo) for combo in sorted(type_combos)]
combo_keys.append("none")

# Générer le dictionnaire
types_to_id = {k: i+1 for i, k in enumerate(combo_keys)}

In [70]:
print(types_to_id)

{'bug': 1, 'bug_dark': 2, 'bug_dragon': 3, 'bug_electric': 4, 'bug_fighting': 5, 'bug_fire': 6, 'bug_flying': 7, 'bug_ghost': 8, 'bug_grass': 9, 'bug_ground': 10, 'bug_ice': 11, 'bug_normal': 12, 'bug_poison': 13, 'bug_psychic': 14, 'bug_rock': 15, 'bug_steel': 16, 'bug_water': 17, 'dark': 18, 'dark_dragon': 19, 'dark_electric': 20, 'dark_fighting': 21, 'dark_fire': 22, 'dark_flying': 23, 'dark_ghost': 24, 'dark_grass': 25, 'dark_ground': 26, 'dark_ice': 27, 'dark_normal': 28, 'dark_poison': 29, 'dark_psychic': 30, 'dark_rock': 31, 'dark_steel': 32, 'dark_water': 33, 'dragon': 34, 'dragon_electric': 35, 'dragon_fighting': 36, 'dragon_fire': 37, 'dragon_flying': 38, 'dragon_ghost': 39, 'dragon_grass': 40, 'dragon_ground': 41, 'dragon_ice': 42, 'dragon_normal': 43, 'dragon_poison': 44, 'dragon_psychic': 45, 'dragon_rock': 46, 'dragon_steel': 47, 'dragon_water': 48, 'electric': 49, 'electric_fighting': 50, 'electric_fire': 51, 'electric_flying': 52, 'electric_ghost': 53, 'electric_grass':

7 : side_conditions et weather : 
-

In [71]:
from poke_env.environment.weather import Weather
from poke_env.environment.side_condition import SideCondition
weather_to_id = {
    None: 0,  # Pour les cas où il n'y a pas de météo
    Weather.UNKNOWN: 0,
    Weather.SUNNYDAY: 1,
    Weather.RAINDANCE: 2,
    Weather.SANDSTORM: 3,
    Weather.HAIL: 4
}
side_condition_to_id = {
    SideCondition.STEALTH_ROCK: 0,
    SideCondition.SPIKES: 1,
    SideCondition.TOXIC_SPIKES: 3,
    SideCondition.REFLECT: 5,
    SideCondition.LIGHT_SCREEN: 7,
    SideCondition.TAILWIND: 9,
    SideCondition.STICKY_WEB: 11  # (pas présent en Gen 4, mais au cas où tu fais d'autres formats)
}

On va modifier un peu ce qu'on a fait. On va créer une espèce de vecteur one-hot pour les conditions : 

["binaire présence/absence de pièges de roc", "spikes", "nb spikes", "t spikes", "nb t spikes", "lightscreen", "nb tours lightscreen", "protection", "nb tours protection", "tailwind', "nb tours tailwind", "sticky"] : vecteur de taille 12.

 Donc on va se servir du dico id pour placer les valeurs aux bons endroits.

In [72]:
#MAJ : tout mettre au format vecteur
#MAJ : ajout du padding 
def ObservedPokemon_to_list(pokemon,is_active_pokemon,is_opponent_team) -> list: 
    # Nom du Pokémon
    species = getattr(pokemon, "species", "UNKNOWN")
    species_id = species_to_id[species]
    item = getattr(pokemon, "item", None)

    item_id = item_to_id[item]
    p = Pokemon(gen=4,species= species)


    # Fraction de vie (float entre 0 et 1)
    hp = getattr(pokemon, "current_hp_fraction", 0.0) or 0.0

    # Types (liste de strings ou [])
    types = [p.type_1.name.lower() if p.type_1 else None, p.type_2.name.lower() if p.type_2 else None]
    if p.type_1 and p.type_2 :
        types = sorted([t for t in types])
    type_str = types[0] if types[1] is None else f"{types[0]}_{types[1]}"
    type_id = types_to_id[type_str]

    # Statut (string ou "NONE")
    status = getattr(pokemon, "status", None)
    status_str = status.name if status else "NONE"
    status_to_id = {
     'NONE': 0,
    'PAR': 1,     # paralysie
    'SLP': 2,     # sommeil
    'BRN': 3,     # brûlure
    'FRZ': 4,     # gel
    'PSN': 5,     # poison
    'TOX': 6,     # poison grave
    'FNT': 7
        }
    status_id = status_to_id[status_str] 

    #Boosts
    if(is_active_pokemon == 1) :

        # Boosts (dict ou [0]*7)
        boosts = getattr(pokemon, "boosts", None)
        boosts_list = list(boosts.values()) if boosts else [0] * 7

    # Stats (dict ou [0]*6)
    if is_opponent_team == 0 : #Si c'est pas l'équipe adverse :

        stats = getattr(pokemon, "stats", None)
        stat_list = list(stats.values()) if stats else [0] * 6
        stats_list = [np.float32(x / 504) if x is not None else 0.0 for x in stat_list]
        #print(f"DEBUUUUUUUUG VALEUR DE STATS_LIST: {stats_list}")

    else : #si c'est l'équipe adverse, on fait autrement
        level = p.level if hasattr(p, "level") and p.level else 80
        hp_stat = floor(((2 * p.base_stats["hp"] + 31 + 84 // 4) * level) / 100) + level + 10
        if hp == None : 
            hp = 1.0
            #On rajoute un quasi-normalize, pour aller autour de 1 sans que ça soit trop petit, donc on prend pas l'exemple leuphorie 714 stats en PV avec lvl 100, nature, EVs
            
        atk = floor((2 * p.base_stats["atk"] + 31 + 84 // 4) * level / 100 + 5) 
        def_ = floor((2 * p.base_stats["def"] + 31 + 84 // 4) * level / 100 + 5) 
        spa = floor((2 * p.base_stats["spa"] + 31 + 84 // 4) * level / 100 + 5) 
        spd = floor((2 * p.base_stats["spd"] + 31 + 84 // 4) * level / 100 + 5) 
        spe = floor((2 * p.base_stats["spe"] + 31 + 84 // 4) * level / 100 + 5) 
        if atk is None :
            atk = 0
        if def_ is None : 
            def_ = 0
        if spa is None : 
            spa = 0
        if spd is None : 
            spd = 0 
        if spe is None : 
            spe = 0
        stats_list = [hp_stat,np.float32(atk/504),np.float32(def_/504),np.float32(spa/504),np.float32(spd/504),np.float32(spe/504)]
        #print(f"DEBUUUUUUUUG VALEUR DE STATS_LIST: {stats_list}")
    #moovepool
    moves_dict = getattr(pokemon, "moves", None)
    move_names = list(moves_dict.keys()) if moves_dict else []

    # Pour avoir toujours 4 valeurs :
    while len(move_names) < 4:
        move_names.append("None")

    move_names_id = [
        moves_to_id[x.lower().replace(" ", "")] if x and x.lower() != "none" else 0 #Padding 0
        for x in move_names
        ]
        
    ability = p.ability
    ability_id = ability_to_id[ability] if ability else 0 # Padding 0 
        
        
    if(is_active_pokemon == 1) :

        return [
            species_id,
            item_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *boosts_list,
            *stats_list,
            *move_names_id[:4]
        ]
    else : 
        return [
            species_id,
            item_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *stats_list,
            *move_names_id[:4]
        ]

In [73]:
#Fonction de padding. Chaque donnée aura la dimension maximale comme ça on aura un vecteur de taille fixe. Et donc quand le vecteur donnée n'atteint pas len_max, on le complète avec des 0 
def pad_list(liste, max_len, pad_value=0.0):
    """Retourne une liste de longueur max_len,
    paddée avec pad_value si besoin."""
    if len(liste) > max_len :
        print(f"[WARN] La liste d'entrée est plus longue ({len(liste)}) que max_len ({max_len}) — elle va être tronquée.")
    res = list(liste[:max_len])
    while len(res) < max_len:
        res.append(pad_value)
    return res

def pad_vector(vector, max_len, pad_value=0.0, dtype=np.float32):
    """Retourne un np.array de longueur max_len,
    paddé avec pad_value si besoin."""
    if len(vector) > max_len :
        print(f"[WARN] Le vecteur d'entrée est plus longue ({len(vector)}) que max_len ({max_len}) — elle va être tronquée.")
    res = list(vector[:max_len])
    while len(res) < max_len:
        res.append(pad_value)
    return np.array(res, dtype=dtype)

In [74]:
#Fonction pour obtenir le weather
def get_weather(dict_observation) :
    obs_weather = dict_observation["weather"]
    if obs_weather:
        weather_enum, turns_left = next(iter(obs_weather.items()))
    else:
        weather_enum, turns_left = None, 0  # ou None, 0 selon ton mapping

    weather = [weather_enum, turns_left]
    return weather


vector conditons : ["binaire présence/absence de pièges de roc", "spikes", "nb spikes", "t spikes", "nb t spikes", "lightscreen", "nb tours lightscreen", "protection", "nb tours protection", "tailwind', "nb tours tailwind", "sticky"] : vecteur de taille 12.

In [75]:
#Fonction pour obtenir les sides conditions
#Remarque : au lieu d'appeller ça conditons on pourrait peut-être appeller ça hazard...
MAX_LEN_SIDE_CONDITIONS = 12
def get_side_conditions(dict_observation,is_my_side) :
    vector = [0 for i in range(MAX_LEN_SIDE_CONDITIONS)] #padding déjà fait du coup
    if is_my_side :
        obs_conditions = dict_observation["side_conditions"]
    else : 
        obs_conditions = dict_observation["opponent_side_conditions"]
    
    if obs_conditions :
        for i in obs_conditions.keys() :
            if i== SideCondition.STEALTH_ROCK or i== SideCondition.STICKY_WEB :
                id = side_condition_to_id[i]
                vector[id] = 1
            else :
                id = side_condition_to_id[i]
                vector[id] = 1
                vector[id+1] = obs_conditions[i] #On prend la valeur associée au hazard dans le dico

    #On normalise car ce vecteur rentre directement dans le réseau profond sans passer par l'embedding
    vector[2] = vector[2]/3 # 3 spikes maximum
    vector[4] = vector[4]/2 # 2 t spikes maximum
    vector[6] = vector[6]/8 # 8 tours de screen max
    vector[8] = vector[8]/8 # 8 tours de screen max
    vector[10] = vector[10]/4 # 4 tours de tailwind max

    return vector

#Verif : j'ai vérifié en lançant des games que ça marchait et ça marchait bien. J'ai pu voir pdr et spikes posé, avec 1 seul spike et c'était bien capturé par ma fonction


Savoir si je suis lock

In [76]:
def is_my_pokemon_locked(dict_observation) :
    poke = dict_observation.get("active_pokemon", None)
    if poke is None:
        return 0  # Pas de pokémon actif, donc pas locké
    item = getattr(poke, "item",None)
    is_choice = item in {"choiceband", "choicescarf", "choicespecs"}

    is_encored = False
    if hasattr(poke, "volatiles") and "encore" in getattr(poke, "volatiles", {}):
        is_encored = True
    elif hasattr(poke, "encore_turns") and getattr(poke, "encore_turns", 0) > 0:
        is_encored = True

    return int(is_choice or is_encored)

In [77]:
#MAX_LEN_SIDE_CONDITIONS = 12, déjà def plus haut
MAX_LEN_WEATHER = 2
LEN_POKE = 16 #C'était pas la bonne valeur car on avait rajouté une feature item donc ça faisait une len de 17, mais vu qu'en pratique on a jamais utilisé 
#ça on a pas vu l'erreur. Bref tkt pas pour les dims c'est 16 pour ceux de la team et 23 pour le poke actif 
LEN_ACTIVE_POKE = 23
MAX_LEN_TEAM_SWITCHES = 80 # 16x5 , cohérent
class adapted_obs_as_readable_str : 
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = get_side_conditions(self.dict_observation,1) #padding et normalisation déjà réalisés dedans

        self.opponent_side_conditions = get_side_conditions(self.dict_observation,0) # pareil padding et normalisation déjà réalisés

        self.weather = get_weather(self.dict_observation)
        self.weather[0] = weather_to_id[self.weather[0]]/4
        self.weather = pad_list(self.weather,max_len =MAX_LEN_WEATHER) 

        self.active_pokemon =list( ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0))
        self.my_team_switches = list(get_my_switches_with_all_infos(battle.available_switches,self.dict_observation))
        self.opponent_active_pokemon = list(ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1)) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = list(get_opponent_team_with_all_infos(self.dict_observation,battle))
        #Fin équipe adverse 
        self.output =   ["side_conditions"] + self.side_conditions + ["opponent_side_conditions"] + self.opponent_side_conditions + ["weather"] + self.weather + ["active_pokemon"] + self.active_pokemon + ["my_team_switches"] + self.my_team_switches + ["opponent_active_pokemon"] + self.opponent_active_pokemon + ["opponent_team"] + self.opponent_team


In [78]:
#on redéfinit obs, pour cette fois-ci avoir tout au format vecteur, et plus rien en str
#MAJ : on le met à jour avec l'obs débuggé str
MAX_LEN_WEATHER = 2
LEN_POKE = 16 
LEN_ACTIVE_POKE = 23 
MAX_LEN_TEAM_SWITCHES = 80 #
IDX_ACTION_SPACE_SIZE = 232
class adapted_obs :
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = np.array(get_side_conditions(self.dict_observation,1),dtype=np.float32) # padding et normalisation déjà réalisés dedans

        self.opponent_side_conditions = np.array(get_side_conditions(self.dict_observation,0),dtype=np.float32) # padding et normalisation déjà réalisés dedans 
        
        self.weather = get_weather(self.dict_observation)
        self.weather[0] = weather_to_id[self.weather[0]]/4
        self.weather = pad_list(self.weather,max_len =MAX_LEN_WEATHER) # padding 
        self.weather = np.array(self.weather,dtype=np.float32)

        self.active_pokemon =np.array( ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0),dtype=np.float32)
        self.my_team_switches = np.array(get_my_switches_with_all_infos(battle.available_switches,self.dict_observation),dtype=np.float32)
        #Calcul du action space avant le padding
        lock = is_my_pokemon_locked(self.dict_observation)
        self.action_space_size = len(self.my_team_switches)//16 + lock*1 + (1-lock)*4 #SI lock alors 1 move, sinon 4
        self.action_space_size = self.action_space_size / 9
        self.action_space_size = np.array([self.action_space_size], dtype=np.float32)
        self.my_team_switches = pad_vector(self.my_team_switches,max_len= MAX_LEN_TEAM_SWITCHES)
        self.opponent_active_pokemon = np.array(ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1),dtype=np.float32) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = np.array(get_opponent_team_with_all_infos(self.dict_observation,battle),dtype=np.float32)
        self.opponent_team = pad_vector(self.opponent_team ,max_len= MAX_LEN_TEAM_SWITCHES)
        #Fin équipe adverse 
        self.output = np.concatenate([
            self.side_conditions, #Fonctionnel
            self.opponent_side_conditions, #Fonctionnel
            self.weather, #Fonctionnel
            self.active_pokemon, #Fonctionnel
            self.my_team_switches, #Fonctionnel
            self.opponent_active_pokemon, #Fonctionnel
            self.opponent_team, #Fonctionnel
            self.action_space_size #Fonctionnel
        ])

#LEN(obs) = 233

Inclus dans ObservedPoke : hp, types, status, moves, ability, stats.
Non inclus : side_conditions, opponent_side_conditions, weather.
Les tailles sont écrites ci-dessous

weather : OK
side_conditions : OK
item : OK
le reste : OK

In [79]:
from poke_env.player.random_player import RandomPlayer

class MyEnvPlayer(Gen4EnvSinglePlayer):
    def __init__(self, *args, **kwargs):
         super().__init__(*args, **kwargs)
     
    def embed_battle(self, battle) -> np.ndarray: #renvoie obs
        #if battle.turn == 1 : 
        obs = adapted_obs(battle) 
        obs = obs.output
        #else : 
            #obs = obs.update(battle)
            
        return obs

    def calc_reward(self, current_battle,last_battle) -> float:
        # Reward simple : +1 si l'adversaire est KO, -1 si toi tu l'es
        reward = 0
        if last_battle.opponent_active_pokemon and current_battle.opponent_active_pokemon.fainted:
            reward += 1
        if last_battle.active_pokemon and current_battle.active_pokemon.fainted:
            reward -= 1
        return reward

    def describe_embedding(self) -> tuple:
        return (2,), np.float32  # 2 valeurs float (my HP, opp HP)
    


In [80]:
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

obs, info = env.reset()
done = False

In [81]:
print(obs[26:26+23])

[ 38.           6.           1.           0.         138.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.58531743   0.41468254
   0.3234127    0.30555555   0.3234127    0.36309522 124.
 177.          27.         118.        ]


In [82]:
############ CODE POUR DEROULER TOUT UN MATCH ###########

# Adversaire
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

# Lancer un match
obs, info = env.reset()
done = False
print(obs)

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 3.6000000e+01 1.0000000e+00 1.0000000e+00 8.6000000e+01
 1.3000000e+01 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.6150794e-01
 4.1666666e-01 2.6388890e-01 2.8174603e-01 4.1666666e-01 3.9682540e-01
 1.7400000e+02 1.1000000e+02 1.8400000e+02 1.5000000e+01 4.9000000e+01
 6.0000000e+00 1.0000000e+00 4.9000000e+01 1.1400000e+02 0.0000000e+00
 5.7142860e-01 4.0476191e-01 4.0476191e-01 4.0476191e-01 4.0476191e-01
 4.0476191e-01 1.2100000e+02 1.6700000e+02 9.1000000e+01 1.7400000e+02
 1.6900000e+02 7.0000000e+00 1.0000000e+00 3.9000000e+01 1.4400000e+02
 0.000

In [83]:
env.close()

Modèle (embedding + réseau profond) :
-


In [84]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

IDX_ACTION_SPACE_SIZE = 232
LEN_POKE = 16 
LEN_ACTIVE_POKE = 23 
LEN_WEATHER = 2
LEN_CONDITIONS = 12
#Réseau perceptron à deux couche cachée, sortie linéaire, f activation = Relu,
class NNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, n_actions)
        #Embedding
        self.item_embedding = nn.Embedding(
        num_embeddings=len(item_to_id) + 1,  # nombre total d'objets
        embedding_dim=8                # dimension du vecteur d'embedding
        )
        self.ability_embedding = nn.Embedding(
        num_embeddings=len(ability_to_id)+1,
        embedding_dim= 8
        )
        self.moves_embedding = nn.Embedding(
        num_embeddings=len(moves_to_id)+1,
        embedding_dim= 16
        )
        self.species_embedding = nn.Embedding(
        num_embeddings=len(species_to_id)+1,
        embedding_dim= 32
        )
        self.types_embedding = nn.Embedding(
        num_embeddings=len(types_to_id)+1,
        embedding_dim= 8
        )
    ####Renvoies le vecteur embedding pour un pokémon####
    def embed_pokemon(self, poke_vec, is_active_poke):
        species  = torch.tensor(int(poke_vec[0]), dtype=torch.long)
        item     = torch.tensor(int(poke_vec[1]), dtype=torch.long)
        ability  = torch.tensor(int(poke_vec[3]), dtype=torch.long)
        types    = torch.tensor(int(poke_vec[4]), dtype=torch.long)
        poke_vec_tensor = torch.tensor(poke_vec, dtype=torch.float32)

        if is_active_poke == 0:
            moves = torch.tensor(poke_vec[11:15], dtype=torch.long)
            mid_vec = poke_vec_tensor[5:11]  # 6 valeurs (status + stats)
        else:
            moves = torch.tensor(poke_vec[18:22], dtype=torch.long)
            mid_vec = poke_vec_tensor[5:18]  # 13 valeurs (status + boosts + stats)
        #print(f"species: {species}, item: {item}, ability: {ability}, types: {types}, moves: {moves}")

        species_emb = self.species_embedding(species)
        item_emb    = self.item_embedding(item)
        ability_emb = self.ability_embedding(ability)
        types_emb   = self.types_embedding(types)
        moves_emb   = self.moves_embedding(moves).flatten()
        hp_tensor   = poke_vec_tensor[2].unsqueeze(0)  # pour concat

        out = torch.cat([species_emb, item_emb, hp_tensor, ability_emb, types_emb, mid_vec, moves_emb], dim=0)
        return out
    
    ####Renvoies le vecteur embedding pour tout le vecteur obs####
    def embedding_obs(self, obs) :
        my_active_poke = obs[12+12+2:12+12+2+23]
        my_other_pokes = [obs[49+16*i:49+16*(i+1)] for i in range(5)]
        my_opponent_active_poke = obs[129:129+23]
        my_opponent_other_pokes = [obs[152+16*i:152+16*(i+1)] for i in range(5)]
        #J'ai récupéré ce que je voulais dans le numpy, maintenant je passe en tensor
        obs_tensor = torch.tensor(obs, dtype=torch.float32)
        obs_conditions_and_weather = obs_tensor[:26]
        obs_action_size = obs_tensor[232].unsqueeze(0)
        #Embeddings
        active_poke_emb = self.embed_pokemon(my_active_poke,is_active_poke = 1)
        my_other_pokes_emb = torch.cat([self.embed_pokemon(my_other_pokes[i],is_active_poke=0) for i in range(5)], dim=0) 
        my_opponent_active_poke_emb = self.embed_pokemon(my_opponent_active_poke,is_active_poke = 1)
        my_opponent_other_pokes_emb = torch.cat([self.embed_pokemon(my_opponent_other_pokes[i],is_active_poke=0) for i in range(5)], dim=0)

        out = torch.cat([obs_conditions_and_weather,active_poke_emb,my_other_pokes_emb,my_opponent_active_poke_emb,my_opponent_other_pokes_emb,obs_action_size])

        return out # TAILLE 1565



    def forward(self, x):
        #Donc x doit être obs de ce que j'ai compris, pas S
        x = self.embedding_obs(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

In [85]:
from poke_env.player.random_player import RandomPlayer

class MyEnvPlayer(Gen4EnvSinglePlayer):
    def __init__(self, *args, **kwargs):
         super().__init__(*args, **kwargs)
     
    def embed_battle(self, battle) -> np.ndarray: #renvoie obs
        #if battle.turn == 1 : 
        obs = adapted_obs(battle) 
        obs = obs.output
        #else : 
            #obs = obs.update(battle)
            
        return obs

    def calc_reward(self, current_battle,last_battle) -> float:
        # Reward simple : +1 si l'adversaire est KO, -1 si toi tu l'es
        reward = 0
        if last_battle.opponent_active_pokemon and current_battle.opponent_active_pokemon.fainted:
            reward += 1
        if last_battle.active_pokemon and current_battle.active_pokemon.fainted:
            reward -= 1
        return reward

    def describe_embedding(self) -> tuple:
        return (2,), np.float32  # 2 valeurs float (my HP, opp HP)

In [86]:
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

obs, info = env.reset()
done = False

In [87]:
############ CODE POUR DEROULER TOUT UN MATCH ###########

# Adversaire
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

# Lancer un match
obs, info = env.reset()
done = False
print(obs)

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 2.8000000e+02 6.0000000e+00 1.0000000e+00 9.7000000e+01
 1.3300000e+02 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.6944442e-01
 3.8293651e-01 3.8293651e-01 2.9563493e-01 2.9563493e-01 4.1865081e-01
 1.8000000e+01 1.4000000e+02 1.0500000e+02 1.3000000e+01 1.5000000e+02
 7.0000000e+00 1.0000000e+00 0.0000000e+00 1.1700000e+02 0.0000000e+00
 5.4563493e-01 2.4603175e-01 3.3333334e-01 4.0079364e-01 4.3452382e-01
 3.3333334e-01 7.8000000e+01 1.1900000e+02 4.3000000e+01 8.1000000e+01
 2.4400000e+02 6.0000000e+00 1.0000000e+00 0.0000000e+00 1.4700000e+02
 0.000

In [88]:
model = NNModel(1565,9)
print(len(obs))
obs_tensor = model.embedding_obs(obs)
print(len(obs_tensor))


233
1565


In [89]:
print(obs_tensor)

tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.3097, -0.2869,  1.0000],
       grad_fn=<CatBackward0>)


In [90]:
print("species max:", len(species_to_id))
print("item max:", len(item_to_id))
print("ability max:", len(ability_to_id))
print("types max:", len(types_to_id))
print("moves max:", len(moves_to_id))

species max: 326
item max: 44
ability max: 102
types max: 154
moves max: 187


Agent : 
-

In [133]:
############## DEFINIR L'EMBEDDING ##################
import time
import random
# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


from poke_env.player.env_player import Gen4EnvSinglePlayer
import numpy as np
import time, random

class Player5Training(Gen4EnvSinglePlayer):
    def __init__(self, model, battle_format="gen4randombattle",max_concurrent_battles=4, **kwargs):
        self._max_concurrent_battles = max_concurrent_battles
        super().__init__(battle_format=battle_format,**kwargs)
        self._init_environment()
        self.model = model
        self.action_space = Discrete(9)

    def _init_environment(self):
            # Hack pour forcer poke-env à prendre en compte le bon nombre de battles
            self._n_battles = self._max_concurrent_battles

    def embed_battle(self, battle):
        # Remplace ça par ta propre obs
        obs = adapted_obs(battle).output
        return obs

    def describe_embedding(self):
        return Box(low=0, high=256, shape=(233,), dtype=np.float32)

    def calc_reward(self, last_battle, current_battle):
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def action_to_move(self, action, battle):
        return super().action_to_move(action, battle)

Adaptation à SB3/Gym et tout le bordel :

Modèle modifié pour que ça marche avec des batchs (c'est un bordel) :
-



In [148]:
class NNModel2(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, n_actions)
        #Embedding
        self.item_embedding = nn.Embedding(
        num_embeddings=len(item_to_id) + 1,  # nombre total d'objets
        embedding_dim=8                # dimension du vecteur d'embedding
        )
        self.ability_embedding = nn.Embedding(
        num_embeddings=len(ability_to_id)+1,
        embedding_dim= 8
        )
        self.moves_embedding = nn.Embedding(
        num_embeddings=len(moves_to_id)+1,
        embedding_dim= 16
        )
        self.species_embedding = nn.Embedding(
        num_embeddings=len(species_to_id)+1,
        embedding_dim= 32
        )
        self.types_embedding = nn.Embedding(
        num_embeddings=len(types_to_id)+1,
        embedding_dim= 8
        )
        self.to(self.device)
    ####Renvoies le vecteur embedding pour un pokémon####
    def embed_pokemon(self, poke_vec, is_active_poke):
        device = self.device
        species  = torch.tensor(int(poke_vec[0]), dtype=torch.long, device=device)
        item     = torch.tensor(int(poke_vec[1]), dtype=torch.long, device=device)
        ability  = torch.tensor(int(poke_vec[3]), dtype=torch.long, device=device)
        types    = torch.tensor(int(poke_vec[4]), dtype=torch.long, device=device)
        #if isinstance(poke_vec, torch.Tensor):
        #   poke_vec_tensor = poke_vec.clone().detach().to(dtype=torch.float32, device=device)
        #else:
        poke_vec_tensor = torch.as_tensor(poke_vec, dtype=torch.float32, device=device)

        if is_active_poke == 0:
            moves = torch.as_tensor(poke_vec[11:15], dtype=torch.long, device=device)
            mid_vec = poke_vec_tensor[5:11]  # 6 valeurs (status + stats)
        else:
            moves = torch.as_tensor(poke_vec[18:22], dtype=torch.long, device=device)
            mid_vec = poke_vec_tensor[5:18]  # 13 valeurs (status + boosts + stats)
        #print(f"species: {species}, item: {item}, ability: {ability}, types: {types}, moves: {moves}")

        species_emb = self.species_embedding(species)
        item_emb    = self.item_embedding(item)
        ability_emb = self.ability_embedding(ability)
        types_emb   = self.types_embedding(types)
        moves_emb   = self.moves_embedding(moves).flatten()
        hp_tensor   = poke_vec_tensor[2].unsqueeze(0)  # pour concat

        out = torch.cat([species_emb, item_emb, hp_tensor, ability_emb, types_emb, mid_vec, moves_emb], dim=0)
        return out
    
    ####Renvoies le vecteur embedding pour tout le vecteur obs####
    def embedding_obs(self, obs_batch):
        # obs_batch : tensor de shape (batch_size, 233)
        batch_size = obs_batch.shape[0]

        # Extraire conditions et météo (mêmes pour chaque batch)
        obs_conditions_and_weather = obs_batch[:, :26]  # (batch_size, 26)

        obs_action_size = obs_batch[:, IDX_ACTION_SPACE_SIZE].unsqueeze(1)  # (batch_size, 1)

        # Extraction des Pokémons
        my_active_pokes = obs_batch[:, 12+12+2 : 12+12+2+23]

        my_other_pokes = [obs_batch[:, 49+16*i : 49+16*(i+1)] for i in range(5)]

        my_opponent_active_pokes = obs_batch[:, 129:129+23]

        my_opponent_other_pokes = [obs_batch[:, 152+16*i : 152+16*(i+1)] for i in range(5)]

        # Embedding (batch)
        active_poke_emb = torch.stack([self.embed_pokemon(my_active_pokes[i], is_active_poke=1) for i in range(batch_size)])

        other_pokes_emb_list = []
        for j in range(5):
            emb_j = torch.stack([self.embed_pokemon(my_other_pokes[j][i], is_active_poke=0) for i in range(batch_size)])
            other_pokes_emb_list.append(emb_j)
        other_pokes_emb = torch.cat(other_pokes_emb_list, dim=1)

        opponent_active_emb = torch.stack([self.embed_pokemon(my_opponent_active_pokes[i], is_active_poke=1) for i in range(batch_size)])

        opponent_other_emb_list = []
        for j in range(5):
            emb_j = torch.stack([self.embed_pokemon(my_opponent_other_pokes[j][i], is_active_poke=0) for i in range(batch_size)])
            opponent_other_emb_list.append(emb_j)
        opponent_other_emb = torch.cat(opponent_other_emb_list, dim=1)

        # Concat final (attention aux bonnes dimensions)
        out = torch.cat([obs_conditions_and_weather, active_poke_emb, other_pokes_emb,
                        opponent_active_emb, opponent_other_emb, obs_action_size], dim=1)

        return out 



    def forward(self, x):
        #Donc x doit être obs de ce que j'ai compris, pas S
        assert x.shape[1] == 1565, f"Taille incorrecte: attendu 1565, reçu {x.shape[1]}"
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

Maintenant que c'est bon : 
-

In [149]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=1565):
        super().__init__(observation_space, features_dim)
        self.nnmodel = NNModel2(input_dim=1565, n_actions=9)
        self._features_dim = features_dim

    def forward(self, observations):
        # observations: tensor shape (batch_size, 233)
        obs = observations.to(self.nnmodel.device)
        #print(f"shape de obs : {obs.shape}")
        x = self.nnmodel.embedding_obs(obs)  # shape (batch_size, 1565)  
        #print(f"shape de x : {x.shape}") 
        return x

In [150]:
from stable_baselines3.common.policies import ActorCriticPolicy

class CustomPolicy(ActorCriticPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, **kwargs):
        super().__init__(
            observation_space,
            action_space,
            lr_schedule,
            features_extractor_class=CustomExtractor,
            **kwargs
        )

Définir l'adversaire (on prend random)

Train
-


Remarque : le vecteur S de taille 1565 est passé à un vecteur de taille 128 avec un nouvel embedding. C'est curieux, pour l'instant on laisse parce que
ça laissera au moins l'entrainement être plus rapide

In [151]:
#from stable_baselines3.common.vec_env import DummyVecEnv
#from stable_baselines3 import PPO
#from stable_baselines3.common.vec_env import SubprocVecEnv

#N_ENVS = 2

#my_model = NNModel2(input_dim=233,n_actions=9)

#def make_env(i):
#    opponent = MyRandomPlayer(i=1+i)  # nom différent pour chaque adversaire
#    return Player5Training(i=i, model=my_model, battle_format="gen4randombattle", opponent=opponent)

#train_envs = SubprocVecEnv([lambda i=i: make_env(i) for i in range(N_ENVS)])


In [152]:
from stable_baselines3.common.vec_env import DummyVecEnv

#my_model = NNModel2(233,9)
#train_env_raw = Player5Training(
#    model=my_model,
#    opponent=RandomPlayer(battle_format="gen4randombattle"),
#    start_challenging=True
#)

#train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

In [153]:
#from stable_baselines3 import PPO
#device="cpu"

#model = PPO(policy=CustomPolicy, batch_size=4, env=train_env, device=device,n_steps=2048)

In [154]:
#model.learn(total_timesteps=1)

On essaie DQN avec ce nouvel obs et embedding pour voir un truc : 
-

In [155]:
from stable_baselines3.dqn.policies import DQNPolicy
from stable_baselines3.common.torch_layers import create_mlp

class CustomDQNPolicy(DQNPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, **kwargs):
        # Forcer les bons paramètres dans kwargs
        kwargs["features_extractor_class"] = CustomExtractor
        kwargs["features_extractor_kwargs"] = {"features_dim": 1565}
        kwargs["net_arch"] = [512, 256, 128]  # ou [] si tout est géré dans CustomExtractor

        super().__init__(
            observation_space,
            action_space,
            lr_schedule,
            **kwargs
        )

In [156]:
from Utils.embedding.old.old_embedding import *
from Utils.embedding.old.type_autoencodeur import TypeAutoencoder
import Players.Player4 as player4

encodeur_type = TypeAutoencoder(encoded_size=4)

# Charger les poids sauvegardés
encodeur_type.load_state_dict(torch.load("Utils/embedding/old/type_autoencoder.pth", map_location='cpu'))
encodeur_type.eval()

class AdversarialTrainPlayer(RandomPlayer): # Adversaire de Player4 lors de l'entrainement

    def __init__(self, model_path="Players/player_4", **kwargs):
        super().__init__(**kwargs)
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        self.model = DQN.load(model_path, device=device)
        print(f"📥 Modèle chargé depuis {model_path}")

    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = player4.init_my_team(battle)
            self.opponent_team, self.opponent_team_index = player4.init_opponent_team(battle)
        else : 
            self.my_team = player4.update_my_team(battle, self.my_team, self.my_team_index)
            self.opponent_team, self.opponent_team_index = player4.update_opponent_team(battle, self.opponent_team, self.opponent_team_index)

        

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        return final_vector
    
    #action_to_move suppr

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action
    
    def choose_move(self, battle):
        obs = self.embed_battle(battle)
        device  = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            action = int(torch.argmax(q_values).item())

        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            return self.choose_random_move(battle)

In [157]:
#opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# Réinstancier l'env
my_model = model
opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")

train_env_raw = Player5Training(
    model = None,
    battle_format="gen4randombattle",
    opponent=opponent,
    max_concurrent_battles=1
)

train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

📥 Modèle chargé depuis Players/player_4


In [158]:
from stable_baselines3 import DQN
#env = train_env

model = DQN(
    policy=CustomDQNPolicy,
    env=train_env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=8,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    verbose=1
)

Using cpu device


In [159]:
#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
#model = DQN.load("Players/player5_nn2", env=train_env, device=device, custom_objects={"policy_class": CustomDQNPolicy})

In [160]:
train_env.reset()
model.learn(total_timesteps=100_000)
model.save("Players/player5_nn2")

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 149      |
|    time_elapsed     | 0        |
|    total_timesteps  | 115      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 159      |
|    time_elapsed     | 1        |
|    total_timesteps  | 251      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 169      |
|    time_elapsed     | 2        |
|    total_timesteps  | 383      |
----------------------------------
----------------------------------
| rollout/          

2025-05-21 17:58:00,529 - AdversarialTrai 10 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Dodrio', 'Pursuit', 'p1a: Rotom', '[from] Pursuit'] in battle battle-gen4randombattle-171886 turn 13
2025-05-21 17:58:00,547 - AdversarialTrai 10 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Dodrio', 'Pursuit', 'p1a: Solrock', '[from] Pursuit'] in battle battle-gen4randombattle-171886 turn 15
2025-05-21 17:58:01,471 - AdversarialTrai 10 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Dialga', 'Outrage', 'p2a: Hippowdon', '[from] lockedmove'] in battle battle-gen4randombattle-171891 turn 13
2025-05-21 17:58:01,641 - AdversarialTrai 10 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Garchomp', 'Outrage', 'p1a: Garchomp', '[from] lockedmove'] in battle battle-gen4randombattle-171892 turn 20


RuntimeError: Placeholder storage has not been allocated on MPS device!

Eval
-

In [ ]:
class Player5Test(Gen4EnvSinglePlayer):
    def __init__(self, model_path="Players/player5_nn2", **kwargs):
        super().__init__(**kwargs)
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        self.model = DQN.load(model_path, device=device )
        print(f"📥 Modèle chargé depuis {model_path}")

    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle):
        obs = adapted_obs(battle)
        return obs.output

    def describe_embedding(self) -> Space:
        return Box(low=np.zeros(233), high=np.full(233, 256), dtype=np.float32)

    def action_to_move(self, action: int, battle: AbstractBattle):
        return super().action_to_move(action, battle)

    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            return int(torch.argmax(q_values).item())

In [ ]:
from tqdm import tqdm

opponent = AdversarialTrainPlayer(battle_format="gen4randombattle", log_level=30)
eval_agent = Player5Test(battle_format="gen4randombattle", opponent= opponent, log_level=30)

n_eval_episodes = 40
wins = 0
rewards = []

obs, _ = eval_agent.reset()
for _ in range(n_eval_episodes):
    done = False
    total_reward = 0

    while not done:
        action = eval_agent.predict(obs)
        obs, reward, terminated, truncated, _ = eval_agent.step(action)
        done = terminated or truncated
        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_agent.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

📥 Modèle chargé depuis Players/player_4
📥 Modèle chargé depuis Players/player5_nn2


/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
2025-05-21 17:39:12,949 - AdversarialTrain 5 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Giratina', 'Outrage', 'p1a: Bibarel', '[from] lockedmove'] in battle battle-gen4randombattle-171663 turn 22
2025-05-21 17:39:17,125 - AdversarialTrain 5 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Vileplume', 'Solar Beam', 'p1a: Manaphy', '[from] lockedmove'] in battle battle-gen4randombattle-171668 turn 25
2025-05-21 17:39:28,944 - AdversarialTrain 5 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Rayquaza', 'Outrage', 'p2a: Purugly', '[from] lockedmove'] in battle battle-gen4randombattle-171683

✅ 1 victoires sur 40 matchs
🎯 Reward moyen : -39.18


100k : 28 victoires sur 40, reward = 16.51
10k : 